# Fill the shared array with data

In [1]:
import numpy as np
import skmap_bindings
import time
import tempfile
from pathlib import Path
from osgeo import gdal, gdal_array
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd

def _model_input(tile, start_year = 2000, end_year = 2022, bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal'], base_url='http://192.168.49.30:8333'):
    prediction_layers = []

    for year in range(start_year, end_year + 1):
        for band in bands:
            prediction_layers += [
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0101_{year}0228_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0301_{year}0430_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0501_{year}0630_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0701_{year}0831_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0901_{year}1031_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}1101_{year}1231_go_epsg.4326_v20230908.tif'
            ]

    return prediction_layers

def _find_connected_indices(raster_files, str1, str2):
    list1 = []
    list2 = []    
    for i, elem1 in enumerate(raster_files):
        if str1 in elem1:
            for j, elem2 in enumerate(raster_files):
                if str2 in elem2 and elem1.split(str1)[-1] == elem2.split(str2)[-1]:
                    list1.append(i)
                    list2.append(j)
                    break                
    return list1, list2

def _raster_paths(df_features, ftype, tile = None, year = None):
    mask = (df_features['type'] == ftype)
    path_col = 'path'
    if ftype == 'landsat':
        df_features['ypath'] = df_features[mask]['path'].apply(lambda p: p.replace('{tile}', tile).replace('{year}', str(year)))
        path_col = 'ypath'
    ids_list = list(df_features[mask]['idx'])
    raster_files = list(df_features[mask][path_col])
    return raster_files, ids_list

def _get_static_layers_info(tiles_gpkg, features_csv, tile):
    tiles = gpd.read_file(tiles_gpkg)
    df_features = pd.read_csv(features_csv)
    min_x, _, _, max_y = tiles[tiles['TILE'] == tile].iloc[0].geometry.bounds
    static_files, _ = _raster_paths(df_features, 'static')
    gidal_ds = gdal.Open(static_files[0]) # It is assumed to be the same for all static layers
    gt = gidal_ds.GetGeoTransform()
    gti = gdal.InvGeoTransform(gt)
    x_off_s, y_off_s = gdal.ApplyGeoTransform(gti, min_x, max_y)
    x_off_s, y_off_s = int(x_off_s), int(y_off_s)
    return static_files, x_off_s, y_off_s

# Parameters setup

In [10]:
tile = '029E_51N'
conf_GDAL = {"CPL_VSIL_CURL_ALLOWED_EXTENSIONS": ".tif",
             "GDAL_HTTP_VERSION": "1.0"}
bands_list = [1,] # The first band is indexed as 1
x_size = 4000
y_size = 4000
n_threads = 96

tiles_gpkg = 'ard2_final_status.gpkg'
features_csv = './model/features.csv'
static_files, x_off_s, y_off_s = _get_static_layers_info(tiles_gpkg, features_csv, tile)
n_static = len(static_files)
idx_static = range(n_static)

x_off_d = 2
y_off_d = 2
subnet = '192.168.49'
dynamic_files = _model_input(tile, start_year = 2000, end_year = 2001, bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal'], base_url=f'http://{subnet}.30:8333')
hosts = [ f'{subnet}.{i}:8333' for i in range(30,43) ]
dynamic_files = [str(r).replace(f"{subnet}.30", f"{subnet}.{30 + int.from_bytes(Path(r).stem.encode(), 'little') % len(hosts)}") for r in dynamic_files]
n_dynamic = len(dynamic_files)
idx_dynamic = range(n_static, n_static+n_dynamic) # row indices of the array where each tiff contenct will go (starting from first element of file_urls)

list_red, list_nir = _find_connected_indices(static_files + dynamic_files, 'red_glad', 'nir_glad')
list_swir1, list_blue = _find_connected_indices(static_files + dynamic_files, 'swir1_glad', 'blue_glad')
list_swir2, _ = _find_connected_indices(static_files + dynamic_files, 'swir2_glad', 'blue_glad')
n_index = len(list_red)
n_indices = 7 * n_index
idx_indices = range(n_static + n_dynamic, n_static + n_dynamic + n_indices)

n_feat = n_static + n_dynamic + n_indices
data = np.empty((n_feat,x_size*y_size), dtype=np.float32, order='C')
print("Parameters setup done")

Parameters setup done


In [11]:
start = time.time()
skmap_bindings.readData(data, n_threads, static_files, idx_static, x_off_s, y_off_s, x_size, y_size, bands_list, conf_GDAL)
print(f"Reading static layers done in {time.time() - start:.2f} s")
print(data)
print(data.shape)

NameError: name 'idx_static' is not defined

In [12]:
print("Starting")
start = time.time()
skmap_bindings.readData(data, n_threads, dynamic_files, idx_dynamic, x_off_d, y_off_d, x_size, y_size, bands_list, conf_GDAL, 255., np.nan)
print(f"Reading dynamic layers done in {time.time() - start:.2f} s")
print(data)
print(data.shape)

Starting
Reading dynamic layers done in 3.64 s
[[2. 0. 2. ... 3. 1. 1.]
 [3. 3. 4. ... 4. 4. 4.]
 [4. 4. 4. ... 4. 5. 5.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(168, 16000000)


In [13]:
len(idx_indices[id_index*n_index:id_index*n_index+n_index])

12

In [14]:
print("Starting")
start = time.time()
# NDVI
band_scaling = 0.004
result_scaling = 125.
result_offset = 125.
id_index = 0
skmap_bindings.computeNormalizedDifference(data, n_threads,
                            list_nir, list_red, idx_indices[id_index*n_index:id_index*n_index+n_index],
                            band_scaling, band_scaling, result_scaling, result_offset)
# NDWI
id_index = 1
skmap_bindings.computeNormalizedDifference(data, n_threads,
                            list_nir, list_swir1, idx_indices[id_index*n_index:id_index*n_index+n_index],
                            band_scaling, band_scaling, result_scaling, result_offset)
# BSI
id_index = 2
skmap_bindings.computeBsi(data, n_threads,
                            list_swir1, list_red, list_nir, list_blue, idx_indices[id_index*n_index:id_index*n_index+n_index],
                            band_scaling, band_scaling, band_scaling, band_scaling, result_scaling, result_offset)
# NDTI
id_index = 3
skmap_bindings.computeNormalizedDifference(data, n_threads,
                            list_swir1, list_swir2, idx_indices[id_index*n_index:id_index*n_index+n_index],
                            band_scaling, band_scaling, result_scaling, result_offset)
# NIRV
id_index = 4
skmap_bindings.computeNirv(data, n_threads,
                            list_red, list_nir, idx_indices[id_index*n_index:id_index*n_index+n_index],
                            band_scaling, band_scaling, result_scaling, result_offset)
# EVI
id_index = 5
skmap_bindings.computeEvi(data, n_threads,
                            list_red, list_nir, list_blue, idx_indices[id_index*n_index:id_index*n_index+n_index],
                            band_scaling, band_scaling, band_scaling, result_scaling, result_offset)
# FAPAR
id_index = 6
skmap_bindings.computeFapar(data, n_threads,
                            list_red, list_nir, idx_indices[id_index*n_index:id_index*n_index+n_index],
                            band_scaling, band_scaling, result_scaling, result_offset)
print(f"C++ ND done in {time.time() - start:.2f} s")
print(data)


Starting
C++ ND done in 2.30 s
[[  2.   0.   2. ...   3.   1.   1.]
 [  3.   3.   4. ...   4.   4.   4.]
 [  4.   4.   4. ...   4.   5.   5.]
 ...
 [204. 204. 203. ... 202. 201. 204.]
 [216. 219. 214. ... 212. 211. 214.]
 [221. 224. 219. ... 223. 220. 220.]]
